In [1]:
#packages
import json
import tqdm
import os
import numpy as np
import sys
import glob
import pandas as pd
import datetime as dt
import re
import math
from collections import defaultdict
import operator

#plot
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
PACKAGE_PARENT = '../'
SCRIPT_DIR = os.path.dirname(os.path.realpath(os.path.join(os.getcwd(), os.path.expanduser('__file__'))))
sys.path.append(os.path.normpath(os.path.join(SCRIPT_DIR, PACKAGE_PARENT)))
from UTILS import preprocessing_Origins, general_cleaning, verification_based_on_initial_record,\
ZoneVariable, HenDailyVariable_Origins, boxplot_distribution_entropy, heatmap_duration_perzone_perhen, is_day
import config_origins as config

C:\Users\camil\anaconda3\lib\site-packages\scipy\sparse\sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


2


In [3]:
#initialise parameter
path_initial_data = config.path_initial_data
path_extracted_data_daily_check = config.path_extracted_data_daily_check
id_run = config.id_run
dico_rc_sys = config.dico_rc_sys
path_FocalBird = config.path_FocalBird

focal_name = config.focal_name
path_Days = config.path_Days

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
#voltage thresholds
#Threshold for minimarker and wintergarden battery change
thresh_dev = 3100
#Threshold for tag battery change
thresh_tag = 2930

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Download data

In [5]:
#for the logs and device updates
over_last_Xdays = 6

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### logs with their associated HenID

In [6]:
p = glob.glob(os.path.join(path_initial_data, 'Barn 4 Pen*\TagUpdates\log*'))
#mtime = os.stat(os.path.join('.', p[0])).st_mtime
#download_time = dt.datetime.fromtimestamp(mtime).strftime('%Y-%m-%d-%H:%M')
#download_time = dt.datetime.strptime(download_time, '%Y-%m-%d-%H:%M')
#print('The file were downloaded at: ', download_time)
    
#print('There is %d log files:\n  %s'%(len(p),'  \n  '.join(p)))
df = preprocessing_Origins(p, config, save=False, dodevice=False)
print(df.shape)
display(df.head(3))
display(df.tail(3))

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<>:1: DeprecationWarning: invalid escape sequence \T
<>:1: DeprecationWarning: invalid escape sequence \T
<ipython-input-6-49996d438290>:1: DeprecationWarning: invalid escape sequence \T
  p = glob.glob(os.path.join(path_initial_data, 'Barn 4 Pen*\TagUpdates\log*'))


From the focalBirdinfo, you have 158 ative tags


C:\Users\camil\Desktop\animals_code\PhD-AnimalWelfare\UTILS.py:409: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_ = df.groupby(['HenID'])['system','PenID','TagID'].agg(lambda x: set(x)).reset_index()


(5688793, 47)


,Timestamp,TagSerialNumber,TagID,Zone,systemgantnerid,useless_zone,signalstrength,zone2,signalstzone2,zone3,...,backpackUniqueID,EPI GENETIC TIMESTAMP,HA 23-11-2020,comportement,weight 23-11-2020,HA 04-01-2021,weight 04-01-2021,HA 01-02-21,weight 01-02-21,weight 12-04-21
0,2020-09-29 09:07:00,02001E13,162,Tier 2 (mini 12),2,8,10,2,8,1,...,12wp,10-date:4-2021,NaN,NaN,"1708,4",the student wrote the weight in kg only one di...,"1800, +70 -30",NaN,"1820,4",1913
120,2020-09-29 09:08:12,02001E6A,140,Tier 2 (mini 11),2,7,10,2,8,1,...,11wp,NaN,NaN,NaN,"1788,9",NaN,"1905,1",NaN,"1905,6","2046,7"
1,2020-09-29 09:08:26,02001E13,162,Tier 2 + untere Stange,2,2,11,8,10,3,...,12wp,10-date:4-2021,NaN,NaN,"1708,4",the student wrote the weight in kg only one di...,"1800, +70 -30",NaN,"1820,4",1913


,Timestamp,TagSerialNumber,TagID,Zone,systemgantnerid,useless_zone,signalstrength,zone2,signalstzone2,zone3,...,backpackUniqueID,EPI GENETIC TIMESTAMP,HA 23-11-2020,comportement,weight 23-11-2020,HA 04-01-2021,weight 04-01-2021,HA 01-02-21,weight 01-02-21,weight 12-04-21
5688048,2021-05-17 08:32:32,020015BB,66,Tier 2 + untere Stange,3,2,11,6,9,3,...,8sps,NaN,NaN,NaN,"1651,2",NaN,"1671,4",NaN,"1709,2","1706,5"
2348622,2021-05-17 08:32:36,02001E69,143,Tier 4 + obere Stange,2,4,19,3,14,2,...,11sb,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"1623,8"
2348399,2021-05-17 08:32:36,02001E14,128,Tier 3 Rampe + Nestbox,2,3,19,2,14,4,...,10ps,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"1921,4","2012,2"


In [7]:
df_FB = pd.read_csv(path_FocalBird, sep=';', parse_dates=['StartDate','EndDate'], dayfirst=True, encoding='latin') #verified dates:correct
#fill end date to today+1 for the birds which we dont know when is there end date (+1: so that today is taken into account)
df_FB['EndDate'].fillna(config.date_max+dt.timedelta(days=1), inplace=True)
df_FB['TagID'] = df_FB['TagID'].astype(str)
df_FB['PenID'] = df_FB['PenID'].map(int).map(str)
#exclude rows were tags were not functionning correctly for some reason 
df_FB = df_FB[df_FB['ShouldBeExcluded']!='yes']
#define a list with the active tags of today/last date
li_active_tags = list(df_FB[df_FB['EndDate']>=config.date_max]['TagID'].unique())
#Counter(li_active_tags)
print('From the focalBirdinfo, you have %d ative tags'%len(li_active_tags))

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


From the focalBirdinfo, you have 158 ative tags


### active tags

In [8]:
df_tag = pd.read_csv(path_FocalBird, sep=';', parse_dates=['StartDate','EndDate'], dayfirst=True, encoding='latin')
df_tag['EndDate'].fillna(dt.date.today(), inplace=True)
print(df_tag.shape)
df_tag = df_tag[df_tag['ShouldBeExcluded']!='yes']
df_tag['TagID'] = df_tag['TagID'].astype(str)
print(df_tag.shape)
df_tag.head(3)

(265, 29)
(250, 29)


C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,HenID,R-Pen,InitialStartDate,ShouldBeExcluded,StartDate,EndDate,FocalLegringID,Expected % of wrong class,Expected chance of wrong treatment,comments,...,TagID,EPI GENETIC TIMESTAMP,HA 23-11-2020,comportement,weight 23-11-2020,HA 04-01-2021,weight 04-01-2021,HA 01-02-21,weight 01-02-21,weight 12-04-21
0,1,1.0,10.06.2020,NaN,2020-09-28,2021-02-01 00:00:00,3,NaN,0,NaN,...,105,9-date:2-2020,NaN,NaN,"1696,5",NaN,"1787,8",NaN,"1800,9",NaN
1,2,1.0,10.06.2020,NaN,2020-09-28,2021-05-17,7,NaN,0,NaN,...,18,NaN,NaN,NaN,"1734,9",NaN,"1691,7",NaN,"1777,8","1782,2"
2,3,1.0,10.06.2020,NaN,2020-09-28,2021-05-17,8,NaN,0,NaN,...,114,NaN,NaN,NaN,"1499,4",NaN,"1714,7",NaN,"1838,3","1887,4"


In [9]:
li_active_tags = list(set(df_tag[df_tag['EndDate']>=dt.date.today()]['TagID'].unique()))
#Counter(li_active_tags)
print('From your focalbird document you should have %d ative tags'%len(li_active_tags))

From your focalbird document you should have 158 ative tags


C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### device

In [10]:
#select last x logs
last_x_logs = math.ceil(over_last_Xdays/2) #arondi en haut
li_logs = []
for path_system in glob.glob(os.path.join(path_initial_data, 'Barn 4 Pen*\DeviceUpdates')):
    li_ = glob.glob(os.path.join(path_system, 'log*'))
    li_.sort(key=lambda x: os.path.getmtime(x), reverse=False)
    li_logs.extend(li_[-last_x_logs:]) 
#next version
#li_newFirmwarecol = ['Timestamp','sort','code','sender','Temperature','Battery Voltage','MovementCounter','LastLFSeen',
#                     'LFCounter','LFRSSISum','Zone', 'Subzone','LFRSSI','Time','LastInfo','LastLocation','LastAction']
li_newFirmwarecol = ['Timestamp','sort','code','sender','Temperature','Battery Voltage','MovementCounter','LastLFSeen',
                     'LFCounter','LFRSSISum','Zone', 'Subzone','Time','LastInfo','LastLocation','LastAction']
#lastReg = LastAction
li_df = []
for path_log in tqdm.tqdm(li_logs):
    #confused month and day
    df_device = pd.read_csv(os.path.join(path_log), sep=';', names=li_newFirmwarecol, parse_dates=['LastAction','Timestamp'],
                           dayfirst=True)
    df_device['data_path'] = path_log
    df_device['system'] = 'pens:'+path_log.split('Barn 4 Pen ')[1].split('\\')[0]
    df_device['logID'] = path_log.split('\\')[-1].split('.')[0]
    print(df_device.shape)
    print('-----------------', path_log)
    #display(df_device.head(3))
    #display(df_device.tail(3))
    li_df.append(df_device)
df_device = pd.concat(li_df)
df_device = df_device[df_device['Timestamp']>=dt.datetime(2020,9,29)]
df['date'] = df['Timestamp'].map(lambda x: dt.datetime.date(x))
df_device['sender'].fillna(' ', inplace=True)
#add the info on if the device needs batterie or not
df_device['NeedsBatteries'] = df_device.apply(lambda x: ('mini' in x['sender']) | (x['sender']=='Wintergarten') | \
                                              (x['sort']=='Tag'), axis=1)
#remove old tags
print(df_device.shape)
df_device = df_device[~((df_device['sort']=='Tag')&(~df_device['sender'].isin(li_active_tags)))]
print(df_device['logID'].unique())
print(df_device.shape)
df_device.tail(3)

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<>:4: DeprecationWarning: invalid escape sequence \D
<>:4: DeprecationWarning: invalid escape sequence \D
<ipython-input-10-f27d1d550a49>:4: DeprecationWarning: invalid escape sequence \D
  for path_system in glob.glob(os.path.join(path_initial_data, 'Barn 4 Pen*\DeviceUpdates')):
 11%|█████████▎                                                                          | 1/9 [00:44<05:52, 44.11s/it]

(214900, 19)
----------------- G:\VPHI\Welfare\2- Research Projects\OFHE2.OriginsE2\GantnerSystem\_dailycheckingSystem\Barn 4 Pen 10 - 12\DeviceUpdates\log_00000016.csv


 22%|██████████████████▋                                                                 | 2/9 [01:28<05:08, 44.13s/it]

(215530, 19)
----------------- G:\VPHI\Welfare\2- Research Projects\OFHE2.OriginsE2\GantnerSystem\_dailycheckingSystem\Barn 4 Pen 10 - 12\DeviceUpdates\log_00000017.csv


 33%|████████████████████████████                                                        | 3/9 [02:17<04:33, 45.65s/it]

...

(219612, 19)
----------------- G:\VPHI\Welfare\2- Research Projects\OFHE2.OriginsE2\GantnerSystem\_dailycheckingSystem\Barn 4 Pen 8 - 9\DeviceUpdates\log_00000012.csv


100%|████████████████████████████████████████████████████████████████████████████████████| 9/9 [05:46<00:00, 38.53s/it]

(124848, 19)
----------------- G:\VPHI\Welfare\2- Research Projects\OFHE2.OriginsE2\GantnerSystem\_dailycheckingSystem\Barn 4 Pen 8 - 9\DeviceUpdates\log_00000013.csv


(1596259, 20)
['log_00000016' 'log_00000017' 'log_00000018' 'log_00000019'
 'log_00000011' 'log_00000012' 'log_00000013']
(1551681, 20)


,Timestamp,sort,code,sender,Temperature,Battery Voltage,MovementCounter,LastLFSeen,LFCounter,LFRSSISum,Zone,Subzone,Time,LastInfo,LastLocation,LastAction,data_path,system,logID,NeedsBatteries
124845,2021-05-17 08:30:44,Tag,02002428,195,28.0,3030.0,52.0,0.0,921.0,7585.0,3.0,2.0,68.0,17.05.2021 08:22:57,17.05.2021 08:30:17,2021-05-17 08:30:17,G:\VPHI\Welfare\2- Research Projects\OFHE2.Ori...,pens:8 - 9,log_00000013,True
124846,2021-05-17 08:30:44,Tag,02002332,196,28.0,2978.0,27.0,2.0,548.0,8296.0,3.0,4.0,255.0,17.05.2021 08:20:44,17.05.2021 06:39:19,2021-05-17 08:28:53,G:\VPHI\Welfare\2- Research Projects\OFHE2.Ori...,pens:8 - 9,log_00000013,True
124847,2021-05-17 08:30:44,Tag,02002333,197,28.0,3003.0,46.0,0.0,979.0,6407.0,3.0,5.0,6.0,17.05.2021 08:28:39,17.05.2021 08:30:20,2021-05-17 08:30:20,G:\VPHI\Welfare\2- Research Projects\OFHE2.Ori...,pens:8 - 9,log_00000013,True


In [11]:
#df_device.groupby(['system','logID'])['NeedsBatteries'].count().reset_index()

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [12]:
#df_device[df_device['logID']=='log_00000010']

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [13]:
#an error might be cause if the alst row was not complet when the file was downloaded

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [14]:
#df_device[df_device['sort']=='Marker'][['code','sender','system']].drop_duplicates() #all actives at least once

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [15]:
#df_device[(df_device['sort']=='Tag')&(df_device['Battery Voltage']<0)]

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Use only the last few days

In [16]:
download_time = max(df_device['Timestamp'].tolist())

#remove the days we dont want
df_device['to_analyse'] = df_device['Timestamp'].map(lambda x: x>=(download_time-dt.timedelta(days=over_last_Xdays)))
df_device = df_device[df_device['to_analyse']]

path_daily_ver = os.path.join(path_extracted_data_daily_check,str(download_time).split(' ')[0])
#create a director if not existing
if not os.path.exists(path_daily_ver):
    os.makedirs(path_daily_ver)

#keep only the last over_last_Xdays days in the logs
df['to_analyse'] = df['Timestamp'].map(lambda x: x>=(download_time-dt.timedelta(days=over_last_Xdays)))
df = df[df['to_analyse']]
df[df['to_analyse']]['date'].unique()    

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array([datetime.date(2021, 5, 11), datetime.date(2021, 5, 12),
       datetime.date(2021, 5, 13), datetime.date(2021, 5, 14),
       datetime.date(2021, 5, 15), datetime.date(2021, 5, 16),
       datetime.date(2021, 5, 17)], dtype=object)

In [17]:
download_time

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Timestamp('2021-05-17 08:30:54')

# Tags last records

In [18]:
#tags last records and active tags with no records
df_ = df[df['TagID'].isin(li_active_tags)].groupby(['TagID','PenID'])['Timestamp'].agg(lambda x: list(x)).reset_index()
df_['last_record_timestamp'] = df_['Timestamp'].map(lambda x: max(x))
df_['nbr_h_since_lastrecord'] = df_['last_record_timestamp'].map(lambda x: int((download_time-x).total_seconds()/60/60))

#add active tags that had no transition at all
li_no_transition = [i for i in li_active_tags if i not in df['TagID'].unique()]
df_no_transition = pd.DataFrame([{'TagID':tag_, 'nbr_h_since_lastrecord':9999} for tag_ in li_no_transition])
print(df_no_transition.shape, df_.shape)
df_ = pd.concat([df_, df_no_transition])
print(df_.shape)
#remove the older row of tags taht appear two times (i.e. if they changed pens for example)
df_.sort_values(['nbr_h_since_lastrecord'], ascending=False, inplace=True)
df_.drop_duplicates(subset=['TagID'], keep='last', inplace=True)
df_[['TagID','PenID','last_record_timestamp','nbr_h_since_lastrecord']].to_csv(os.path.join(path_daily_ver, 
                                                                           id_run+str(download_time).split(' ')[0]+'h_TagsLastRecords.csv'), 
                                                              sep=';', index=False)

h = 10
print('------------- Tags with no records since at least %d hours: -------------'%h)
print('There is %d such tags, we will print a maximum of 50 line (starting form the worst ones)'%df_[df_['nbr_h_since_lastrecord']>h].shape[0])
li_tag_issues = df_[df_['nbr_h_since_lastrecord']>h][['TagID','PenID','last_record_timestamp','nbr_h_since_lastrecord']]['TagID'].tolist()
print('There is %d tags with at least one issue'%len(set(li_tag_issues)))
df_tag_summary = df_.copy()
df_[df_['nbr_h_since_lastrecord']>h][['TagID','PenID','last_record_timestamp','nbr_h_since_lastrecord']].head(50)

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(0, 0) (158, 5)
(158, 5)
------------- Tags with no records since at least 10 hours: -------------
There is 0 such tags, we will print a maximum of 50 line (starting form the worst ones)
There is 0 tags with at least one issue


,TagID,PenID,last_record_timestamp,nbr_h_since_lastrecord


In [19]:
#df[df['TagID']=='19'] #10.04 morning: verify in video
#df_device[df_device['sender']=='29']

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Reader last update

#understand how the device logs are
df_test = df_device.groupby('code')['system'].agg(lambda x: len(set(x))).reset_index()
df_test[df_test['system']>1] #--> no code in two system!!
#importantly:
df_device[df_device['sort']!='Tag'].groupby(['sender'])['system','code','sort'].agg(lambda x: set(x)).reset_index()

In [20]:
for reader_code in dico_rc_sys.keys():
    print("The last read out from the reader of %s arrived at: %s"%(dico_rc_sys[reader_code],
                                                    str(max(df_device[df_device['code']==reader_code]['LastAction'].tolist()))))

The last read out from the reader of Reader Pen 3-5 arrived at: 2021-05-17 08:30:52
The last read out from the reader of Reader Pen 8-9 arrived at: 2021-05-17 08:30:36
The last read out from the reader of Reader Pen 10-12 arrived at: 2021-05-17 08:29:21


C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#cleaning step: remove days for specific hens when the tag was not sending output regularly anymore
df_test = df_device[df_device['sort']=='Tag'].copy()
df_test['date'] = df_test['Timestamp'].map(lambda x: dt.datetime.date(x))
df_test = df_test.groupby(['sender','date','hour'])['LastAction'].agg(lambda x: list(x)).reset_index()
df_test.rename(columns={'sender':'TagID'}, inplace=True)
df_test['nbr_last_action']
#last action biggest gap
print(df_test.shape)
df_test.head(3)

## LF Counter

In [21]:
df_LFCoutner = df_device[df_device['sort']=='Tag'].copy()
df_LFCoutner['date'] = df_LFCoutner['Timestamp'].map(lambda x: dt.datetime.date(x))
#df_LFCoutner['is_day'] = df_LFCoutner['Timestamp'].map(lambda x: is_day(x, config.dico_night_hour))
df_LFCoutner = df_LFCoutner.groupby(['sender','date'])['LFCounter'].agg(lambda x: sorted(list(x))).reset_index()
df_LFCoutner['LFCounter_nbr_equal0'] = df_LFCoutner['LFCounter'].map(lambda x: sum([i==0 for i in x]))
df_LFCoutner['LFCounter_atleastone0'] = df_LFCoutner['LFCounter_nbr_equal0'].map(lambda x: x>0)
print(df_LFCoutner.shape)
df_ = df_LFCoutner[df_LFCoutner['LFCounter_atleastone0']].groupby(['sender'])[['date',
                                                                'LFCounter_nbr_equal0']].agg(lambda x: list(x)).reset_index()
df_.to_csv(os.path.join(path_daily_ver, id_run+str(download_time).split(' ')[0]+'_LFCounterEqual0.csv'),sep=';')
df_

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(1106, 5)


,sender,date,LFCounter_nbr_equal0
0,108,[2021-05-15],[1]
1,114,"[2021-05-14, 2021-05-16]","[2, 1]"
2,140,[2021-05-12],[1]
3,16,"[2021-05-12, 2021-05-14]","[1, 1]"
4,170,"[2021-05-11, 2021-05-12, 2021-05-13]","[3, 1, 4]"
5,176,"[2021-05-12, 2021-05-14, 2021-05-15]","[1, 2, 5]"
6,178,[2021-05-14],[1]
7,18,"[2021-05-15, 2021-05-16]","[1, 2]"
8,182,[2021-05-14],[2]
9,19,[2021-05-12],[1]


# Device Last update

In [22]:
#TODO
#New data!!!: plt.hist(df_device['MovementCounter'].dropna(),bins='auto');
#plt.hist(df_device['Temperature'].dropna(),bins=30);

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## biggest gap over the entire day

In [23]:
df_lastactionGAP = df_device[df_device['sort']=='Tag'].copy()
df_lastactionGAP['date'] = df_lastactionGAP['Timestamp'].map(lambda x: dt.datetime.date(x))
#df_lastactionGAP['is_day'] = df_lastactionGAP['Timestamp'].map(lambda x: is_day(x, config.dico_night_hour))
#df_lastactionGAP = df_lastactionGAP[df_lastactionGAP['is_day']]
df_lastactionGAP = df_lastactionGAP.groupby(['sender','date'])['LastAction'].agg(lambda x: sorted(list(x))).reset_index()
#last action biggest gap of the day
df_lastactionGAP['li_gap_of_the_day_tsstarted'] = df_lastactionGAP['LastAction'].map(lambda x: {x[i]:(x[i+1]-x[i]).total_seconds() for i in range(0,len(x)-1)})
df_lastactionGAP['biggest_gap_of_day_mn'] = df_lastactionGAP['li_gap_of_the_day_tsstarted'].map(lambda x: round(max(x.items(), 
                                                                                         key=operator.itemgetter(1))[1]/60,0))
df_lastactionGAP['start_biggest_gap_of_day'] = df_lastactionGAP['li_gap_of_the_day_tsstarted'].map(lambda x: max(x.items(), 
                                                                                               key=operator.itemgetter(1))[0])
df_lastactionGAP.to_csv(os.path.join(path_daily_ver, id_run+str(download_time).split(' ')[0]+'_LastactionGAP.csv'),sep=';')
print(df_lastactionGAP.shape)
df_lastactionGAP[df_lastactionGAP['biggest_gap_of_day_mn']>12]

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(1106, 6)


,sender,date,LastAction,li_gap_of_the_day_tsstarted,biggest_gap_of_day_mn,start_biggest_gap_of_day
42,108,2021-05-11,"[2021-05-11 08:28:55, 2021-05-11 08:40:50, 202...","{2021-05-11 08:28:55: 715.0, 2021-05-11 08:40:...",13.0,2021-05-11 13:17:47
59,114,2021-05-14,"[2021-05-14 00:00:07, 2021-05-14 00:10:14, 202...","{2021-05-14 00:00:07: 607.0, 2021-05-14 00:10:...",88.0,2021-05-14 17:31:09
61,114,2021-05-16,"[2021-05-15 23:59:50, 2021-05-16 00:10:00, 202...","{2021-05-15 23:59:50: 610.0, 2021-05-16 00:10:...",13.0,2021-05-16 17:35:46
69,115,2021-05-17,"[2021-05-17 00:09:34, 2021-05-17 00:19:37, 202...","{2021-05-17 00:09:34: 603.0, 2021-05-17 00:19:...",14.0,2021-05-17 00:33:43
605,198,2021-05-14,"[2021-05-14 00:08:50, 2021-05-14 00:18:59, 202...","{2021-05-14 00:08:50: 609.0, 2021-05-14 00:18:...",14.0,2021-05-14 10:24:54
1074,93,2021-05-14,"[2021-05-14 00:00:25, 2021-05-14 00:10:32, 202...","{2021-05-14 00:00:25: 607.0, 2021-05-14 00:10:...",13.0,2021-05-14 02:38:13


### very last update

In [24]:
#sort by date in order to take the last voltage using a simple list
df_ = df_device.groupby(['code','sender','system','sort','NeedsBatteries'])['LastAction'].agg(lambda x: list(x)).reset_index()
df_['last_LastAction'] = df_['LastAction'].map(lambda x: max(x))
df_['nbr_mn_since_LastUpdate'] = df_['last_LastAction'].map(lambda x: (download_time-x).total_seconds()/60)

#add tags with no update:
li_no_transition = [i for i in li_active_tags if i not in df_device['sender'].unique()]
df_no_transition = pd.DataFrame([{'sender':tag_, 'nbr_mn_since_LastUpdate':9999} for tag_ in li_no_transition])
print(df_no_transition.shape, df_.shape)
df_ = pd.concat([df_, df_no_transition])
print(df_.shape)
print('There is %d device with 0 updates'%df_no_transition.shape[0])
df_.sort_values(['nbr_mn_since_LastUpdate'], ascending=False, inplace=True)
#remove the older sender that appear two times (i.e. if they changed pens for example)
df_.drop_duplicates(subset=['system','sender'], keep='last', inplace=True)
df_[['code','sender','system','sort','last_LastAction','nbr_mn_since_LastUpdate']].to_csv(os.path.join(path_daily_ver, 
                                            id_run+str(download_time).split(' ')[0]+'h_DeviceLastUpdate.csv'), sep=';', index=False)


mn = 12
li_tag_issues.extend(df_[df_['nbr_mn_since_LastUpdate']>mn]['sender'])
print('There is %d tags with at least one issue'%len(set(li_tag_issues)))

print('------------- Device with no update since at least %d minutes -------------'%mn)
df_tag_summary = pd.merge(df_tag_summary, df_[df_['sort']=='Tag'], left_on='TagID', right_on='sender', how='outer')
#display(df_tag_summary.head(3))
display(df_[df_['nbr_mn_since_LastUpdate']>mn][['code','sender','system','sort','last_LastAction','nbr_mn_since_LastUpdate']])

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(0, 0) (185, 8)
(185, 8)
There is 0 device with 0 updates
There is 0 tags with at least one issue
------------- Device with no update since at least 12 minutes -------------


,code,sender,system,sort,last_LastAction,nbr_mn_since_LastUpdate


In [25]:
download_time

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Timestamp('2021-05-17 08:30:54')

# Critical Voltage last D days

In [26]:
D = 7 #assuming I do this one a week, it make sense to plot it once a week
#D=2 was not enough as we would miss some tags that were at 2600 (e.g. 82)
df_device['is_last_'+str(D)+'days'] = df_device['Timestamp'].map(lambda x: x>=(download_time-dt.timedelta(days=D)))
df_ = df_device[df_device['is_last_'+str(D)+'days']].groupby(['code','sender','system','sort',
                                                        'NeedsBatteries'])['Battery Voltage'].agg(lambda x: list(x)).reset_index()
df_['min_Battery_Voltage_last_'+str(D)+'days'] = df_['Battery Voltage'].map(lambda x: min(x))
df_['avg_Battery_Voltage_last_'+str(D)+'days'] = df_['Battery Voltage'].map(lambda x: np.nanmean(x))
df_.sort_values(['min_Battery_Voltage_last_'+str(D)+'days'], ascending=False, inplace=True)
# check whether at least one of the last entries was below the critical threshold 
df_['CRITICAL'] = df_.apply(lambda x:sum([i<thresh_dev for i in x['Battery Voltage']])>=1 if x['sort']!='Tag' \
                            else sum([i<thresh_tag for i in x['Battery Voltage']])>=1, axis=1)
df_.to_csv(os.path.join(path_daily_ver,id_run+str(download_time).split(' ')[0]+'h_Battery VoltageLast'+str(D)+'days.csv'), sep=';', index=False)
print('------------- Critical Battery Voltage of device/tags (under threshold at least once during the last %d days -------------'%D)
li_tag_issues.extend(df_[(df_['CRITICAL'])&(df_['sort']=='Tag')]['sender'])
print(df_tag_summary.shape)
df_tag_summary = df_tag_summary.drop(['sender'], axis=1)
df_tag_summary = pd.merge(df_tag_summary, df_[df_['sort']=='Tag'][['CRITICAL','min_Battery_Voltage_last_'+str(D)+'days',
                                                                   'avg_Battery_Voltage_last_'+str(D)+'days','sender']], 
                          left_on='TagID', right_on='sender', how='outer')
df_tag_summary.drop(['LastAction','Timestamp','NeedsBatteries'], axis=1).to_csv(os.path.join(path_daily_ver,id_run+str(download_time).split(' ')[0]+'_TagSummary.csv'), sep=';', index=False)
print(df_tag_summary.shape)
#display(df_tag_summary.head(3))
print('There is %d tags with at least one issue:'%len(set(li_tag_issues)))
df_[(df_['CRITICAL'])&(df_['NeedsBatteries'])]

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<ipython-input-26-207c442b845e>:7: RuntimeWarning: Mean of empty slice
  df_['avg_Battery_Voltage_last_'+str(D)+'days'] = df_['Battery Voltage'].map(lambda x: np.nanmean(x))


------------- Critical Battery Voltage of device/tags (under threshold at least once during the last 7 days -------------
(158, 13)
(158, 16)
There is 19 tags with at least one issue:


,code,sender,system,sort,NeedsBatteries,Battery Voltage,min_Battery_Voltage_last_7days,avg_Battery_Voltage_last_7days,CRITICAL
77,020015CB,86,pens:8 - 9,Tag,True,"[2969.0, 2969.0, 2967.0, 2971.0, 2971.0, 2969....",2929.0,2968.778035,True
26,02001512,12,pens:3 - 5,Tag,True,"[3005.0, 3008.0, 3009.0, 3007.0, 3009.0, 3007....",2926.0,3009.265896,True
103,02001E25,131,pens:10 - 12,Tag,True,"[2992.0, 2992.0, 2995.0, 2992.0, 2991.0, 2992....",2917.0,2987.842593,True
83,0200196B,122,pens:10 - 12,Tag,True,"[2943.0, 2939.0, 2945.0, 2940.0, 2940.0, 2946....",2912.0,2934.515046,True
8,02001076,76,pens:8 - 9,Tag,True,"[2983.0, 2982.0, 2980.0, 2979.0, 2979.0, 2982....",2906.0,2974.309827,True
30,0200151F,73,pens:3 - 5,Tag,True,"[2979.0, 2977.0, 2979.0, 2980.0, 2978.0, 2977....",2900.0,2980.470520,True
117,02001E3B,149,pens:10 - 12,Tag,True,"[2986.0, 2984.0, 2985.0, 2988.0, 2991.0, 2987....",2900.0,2983.380787,True
129,02001E71,151,pens:10 - 12,Tag,True,"[2997.0, 2999.0, 2995.0, 2995.0, 2993.0, 3002....",2899.0,2995.453704,True
49,02001558,58,pens:3 - 5,Tag,True,"[2961.0, 2962.0, 2962.0, 2962.0, 2958.0, 2961....",2867.0,2962.085549,True
16,0200141B,47,pens:3 - 5,Tag,True,"[2975.0, 2971.0, 2967.0, 2966.0, 2965.0, 2971....",2866.0,2967.456647,True


#plot

df_device = df_device.sort_values(['LastAction'])
for tag in li_active_tags:
    df_plt = df_device[df_device['sender']==str(tag)][['voltage','Timestamp']]
    plt.plot(df_plt['Timestamp'].tolist(), df_plt['voltage'].tolist());
    plt.title(tag)
    plt.show()

# Plot Tags

#plot the batterie level, the tagupdates and the number of transition for each tags, one plot per day
df_device['date'] = df_device['Timestamp'].map(lambda x: dt.datetime.date(x))
df_device = df_device.sort_values(['LastAction'])
#us the log files to crate a df_tr file with one row per hen per hour, and the number of transition in a column
df['hour'] = df['Timestamp'].map(lambda x: x.hour if pd.isnull(x)!=np.nan else np.nan)
df_tr = df.groupby(['TagID','date','hour'])['Timestamp'].count().reset_index()
df_tr.rename(columns={'Timestamp':'nbr_transition_last1h'}, inplace=True)
df_tr['date_hour'] = df_tr.apply(lambda x: dt.datetime(x['date'].year, x['date'].month, x['date'].day, x['hour'],0,0), axis=1)

#for tag in tqdm.tqdm(set(li_tag_issues)):
for tag in tqdm.tqdm(li_active_tags):
    try:
        plt.figure(figsize=(18,4))
        ax = plt.subplot(111) 

        #Battery Voltage
        df_plt = df_device[df_device['sender']==tag].copy()
        SystemID = df_plt['system'].iloc[0]
        ax.plot(df_plt['Timestamp'].tolist(), df_plt['Battery Voltage'].tolist(), color='blue');

        #number of transition in the last hours    
        ax2=ax.twinx()
        df_tr_ = df_tr[df_tr['TagID']==tag].copy()
        ax2.plot(df_tr_['date_hour'].tolist(), df_tr_['nbr_transition_last1h'].tolist(), color='green');

        #number of device update in the last hour
        df_plt['hour'] = df_plt['Timestamp'].map(lambda x: x.hour if pd.isnull(x)!=np.nan else np.nan)
        df_up = df_plt.groupby(['date','hour'])['LastAction'].agg(lambda x: list(x)).reset_index()
        df_up['date_hour'] = df_up.apply(lambda x: dt.datetime(x['date'].year, x['date'].month, x['date'].day, x['hour'],0,0), axis=1)
        df_up['nbr_update_last1h'] = df_up.apply(lambda x: sum([i>=(x['date_hour']-dt.timedelta(hours=1)) for i in x['LastAction']]), 
                                                 axis=1)
        ax2.plot(df_up['date_hour'].tolist(), df_up['nbr_update_last1h'].tolist(), color='r');

        #add days and plot
        for day in df_plt['date'].unique() : 
            plt.axvline(x=day, linewidth=1, color='b')
        plt.title('TagID: '+tag)
        #for label in ax.get_xticklabels():
        #    label.set_rotation(90) 
        y_ = np.nanmax(df_plt['Battery Voltage'].tolist())
        x_ = np.nanmin(df_plt['date'].unique())
        ax.text(x_, y_-0.1*(y_-np.nanmin(df_plt['Battery Voltage'].tolist())), '#device update in the last hour -right axis', fontsize=10, color='r')
        ax.text(x_, y_-0.2*(y_-np.nanmin(df_plt['Battery Voltage'].tolist())), '#transition in the last hour -right axis', fontsize=10, color='green')
        ax.text(x_, y_-0.3*(y_-np.nanmin(df_plt['Battery Voltage'].tolist())), 'Battery Voltage -left axis', fontsize=10, color='blue')

        #create a director if not existing
        path_incase = os.path.join(path_daily_ver, 'INcaseALLplot', SystemID.replace(':',''))
        if not os.path.exists(path_incase):
            os.makedirs(path_incase)
        plt.savefig(os.path.join(path_incase,id_run+'_'+str(tag)+'Tag_CHECK.png'), dpi=300,
                                format='png', bbox_inches='tight')
    except Exception as e:
                print('ERROR:-------------',e, '    TAG: ',tag)
    #plt.show()